**STEP#01**

insert distinct list of patient and facility ID

Iteration 1:          22,754,008

Iteration 2:          27,476,749

In [ ]:
SELECT	[pat_national_id]
	,	[facility_id]
	,	COUNT(*)    RN
INTO [Mawid].[dbo].[cas_appointment_STP_1]
FROM    [Mawid].[dbo].[cas_appointment]
GROUP BY [pat_national_id]
    ,    [facility_id];

Badriya update facility\_CD 69,000,000 88,253,309

In [ ]:
INSERT INTO [Mawid].[dbo].[cas_appointment_STP_1] ([pat_national_id],	[facility_id], RN)
SELECT	[pat_national_id]
	,	SUBSTRING([appt_code],11,14) AS [facility_id]
	,	COUNT(*)    RN
FROM [Mawid].[dbo].[cas_appointment]
GROUP BY SUBSTRING([appt_code],11,14)
      ,[pat_national_id]

STEP#02 Insert patient from PATIENT table into stp-1 where they have  a default PHC 2,987,289 2,987,289

In [ ]:
INSERT INTO [Mawid].[dbo].[cas_appointment_STP_1] ([pat_national_id]
	,	[facility_id])
SELECT	STR([pat_national_id])		[pat_national_id]
	,	default_phc					[facility_id]
FROM	[Mawid].[dbo].[cas_patient]
WHERE default_phc IS NOT NULL

  --118,717,347

\--STEP#03 remove dublication 23,840,809 116,734,516

In [ ]:
SELECT	pat_national_id
	,	facility_id
INTO [Mawid].[dbo].[cas_appointment_STP_2]
FROM	[Mawid].[dbo].[cas_appointment_STP_1]
GROUP BY pat_national_id
	,	facility_id;

STEP#04 get MOH\_ID 23,840,809 116,734,516

In [ ]:
WITH	facility	AS	(	SELECT	[facility_description]
								,	facility_name
								,   [facility_description_ar]
								,   CAST(CAST([facility_cd]  AS FLOAT) AS NVARCHAR(100)) [facility_cd]
								,   [moh_facility_id]
								,   [cas_facility_type_id]
							FROM --[Mawid].[dbo].[cas_facility]
							[Mawid].[dbo].[facility_list_210322])
SELECT	pat_national_id
	,	facility_id
	,	f.moh_facility_id
INTO [Mawid].[dbo].[cas_appointment_STP_3]
FROM	[Mawid].[dbo].[cas_appointment_STP_2] a
LEFT JOIN facility		f	ON a.facility_id		= f.facility_cd;

STEP#05 get CLUSTER value 23,840,809 116,750,846

In [ ]:
SELECT	DISTINCT
		pat_national_id
	,	facility_id
	,	f.moh_facility_id
	,	h.CLUSTER
INTO [Mawid].[dbo].[cas_appointment_STP_4]
FROM	[Mawid].[dbo].[cas_appointment_STP_3] f
LEFT JOIN	[PatientInfo].[dbo].[HospitalList_Updated]	h	ON f.moh_facility_id	= h.MOHID;


STEP#06 Remove null cluster values, & aggregate cluster list 9,950,325

In [ ]:
SELECT	pat_national_id							--16,794,780
	,	CLUSTER
INTO [Mawid].[dbo].[cas_appointment_STP_51] 
FROM	[Mawid].[dbo].[cas_appointment_STP_4] 
WHERE  CLUSTER IS NOT NULL
GROUP BY pat_national_id,CLUSTER;

--INSERT NVR	
INSERT INTO [Mawid].[dbo].[cas_appointment_STP_51] (pat_national_id, CLUSTER)	--8,090,495
SELECT	[personalidentifier]	AS pat_national_id
	,	Cluster	AS CLUSTER
FROM	[PHAP_EDWH].[NVR].[Total_Vaccines_30082021_14_51] N
LEFT JOIN [BI_TestDB].[dbo].[NVR Region List] R ON N.region_en = R.region_en 
												AND N.city_en = R.city_en
WHERE Cluster IS NOT NULL

--Remove dublication
SELECT pat_national_id					--22,093,232
	, CLUSTER
INTO [Mawid].[dbo].[cas_appointment_STP_52]
FROM [Mawid].[dbo].[cas_appointment_STP_51] 
GROUP BY pat_national_id
	, CLUSTER

SELECT	pat_national_id					--14,482,562
	,	STRING_AGG(CLUSTER, ',')	CLUSTER_LIST
INTO [Mawid].[dbo].[cas_appointment_STP_5] 
FROM	[Mawid].[dbo].[cas_appointment_STP_52] 
WHERE  CLUSTER IS NOT NULL
GROUP BY pat_national_id;

STEP#07 Remove null cluster values 9,950,325 14,482,562

In [ ]:
WITH CTE AS (	SELECT	STR([pat_national_id]) AS [pat_national_id]
					,	[pat_date_of_birth]
					,	gender_id
				FROM	[Mawid].[dbo].[cas_patient]
				WHERE [pat_national_id] != 0)
SELECT	a.[pat_national_id]
	,	CASE	WHEN gender_id = 1 THEN 'M' 
				WHEN gender_id = 2 THEN 'F' 
				ELSE 'U' END AS gender_id
	,	pat_date_of_birth
	,	CLUSTER_LIST
INTO [Mawid].[dbo].[cas_appointment_STP_6]
FROM		[Mawid].[dbo].[cas_appointment_STP_5]	a
LEFT JOIN	CTE	ON a.pat_national_id	= CTE.pat_national_id;

STEP#08 Get shared dataset between MAWID & NHIC 9,330,676 14,367,049

In [ ]:
SELECT	m.*
INTO	[Mawid].[dbo].[cas_appointment_STP_7]
FROM	[Mawid].[dbo].[cas_appointment_STP_6] m
INNER JOIN [PHAP_EDWH].[NHIC].[EXPORT_20210825] n ON m.pat_national_id = STR(n.ID);

STEP#08 Get shared dataset between MAWID & NHIC 9,330,676 14,367,049

In [ ]:
SELECT	[pat_national_id]														AS	[ID]
	,	gender_id																AS	[GENDER]
	,	FORMAT(DATEDIFF(hour,[pat_date_of_birth],GETDATE())/8766.0, 'n2')		AS	[AGE]
	,	CASE	WHEN [pat_national_id] LIKE '1%' THEN 'Saudi' 
				WHEN [pat_national_id] LIKE '2%' THEN 'non-Saudi' ELSE NULL END	AS	[is_saudi]
	,	CASE	WHEN DATEDIFF(HOUR,[pat_date_of_birth],GETDATE())/8766.0 <= 1  THEN 'Infant' 
				WHEN DATEDIFF(HOUR,[pat_date_of_birth],GETDATE())/8766.0 <= 4  THEN 'Toddler'
				WHEN DATEDIFF(HOUR,[pat_date_of_birth],GETDATE())/8766.0 <= 12 THEN 'Child ' 
				WHEN DATEDIFF(HOUR,[pat_date_of_birth],GETDATE())/8766.0 <= 19 THEN 'Teen' 
				WHEN DATEDIFF(HOUR,[pat_date_of_birth],GETDATE())/8766.0 <= 39 THEN 'Adult' 
				WHEN DATEDIFF(HOUR,[pat_date_of_birth],GETDATE())/8766.0 <= 59 THEN 'Middle Age Adult' 
				WHEN DATEDIFF(HOUR,[pat_date_of_birth],GETDATE())/8766.0 >  59 THEN 'Senior Adult' 
				ELSE NULL END													AS [Age_Class]
	,	CASE	WHEN POLICYEXPIRYDATE > GETDATE() then 'Active' 
				WHEN POLICYEXPIRYDATE < GETDATE() then 'Expired' 
				ELSE 'NA' END													AS	[Insurance_Status]
	,	CLUSTER_LIST
	,	pat_date_of_birth
INTO	[Mawid].[dbo].[MAWID_INFO]
FROM	[Mawid].[dbo].[cas_appointment_STP_7] benf
LEFT JOIN [PHAP_EDWH].[ORG].[CCHI_Data]  cchi ON benf.pat_national_id = cchi.IDENTITYNUMBER

STEP#08 Get BENFICIARY list with CLuster List 3,485,922

In [ ]:
SELECT	[NIN]				--4,229,633
	,	Cluster
INTO [Mawid].[dbo].[BENF_ClusterList_1]
FROM	[PHAP_EDWH].[BNF].[BENF_Visits]
GROUP BY [NIN]
	,	Cluster


SELECT	[NIN]				--3,905,099
	,	STRING_AGG(Cluster, ',') Cluster_List
INTO [Mawid].[dbo].[BENF_ClusterList]
FROM [Mawid].[dbo].[BENF_ClusterList_1]
WHERE Cluster IS NOT NULL
GROUP BY [NIN];



WITH	CTE_M	AS	(	SELECT	ID	AS	NIN
							,	value	CLUSTER
						FROM	[Mawid].[dbo].[MAWID_INFO]
						CROSS APPLY STRING_SPLIT(CLUSTER_LIST, ',')
						WHERE value = 'R3')
	,	CTE_B	AS	(	SELECT	NIN
							,	value	CLUSTER
						FROM	[Mawid].[dbo].[BENF_ClusterList]
						CROSS APPLY STRING_SPLIT(CLUSTER_LIST, ',')
						WHERE value = 'R3')
SELECT	FORMAT(COUNT(*), 'n0')	COUNT_DIFF
FROM	CTE_B	a
WHERE	NIN	 NOT IN (SELECT NIN FROM CTE_B)
--WHERE	NIN	IN (SELECT NIN FROM CTE_B)		
GROUP BY a.Cluster

--SELECT	FORMAT(COUNT(*), 'n0')	AS	NIN
--	,	value	CLUSTER
--FROM	[Mawid].[dbo].[cas_appointment_STP_8]
--CROSS APPLY STRING_SPLIT(CLUSTER_LIST, ',')
--GROUP BY value

--SELECT	FORMAT(COUNT(*), 'n0')
--	,	value	CLUSTER
--FROM	[PHAP_EDWH].[BNF].[BENF_ClusterList]
--CROSS APPLY STRING_SPLIT(CLUSTER_LIST, ',')
--GROUP BY value

**Cluster BENF MAWID Mawid Not BENF Mawif & BENF Benf Not Mawid**

\===============================================================================

R1      1,241,236   1,470,987   1,401,487 69,500        1,171,736

R2      1,150,146   831,736     781,311        50,425        1,099,721

R3      809,914     822,992     810,312        12,680        797,234

E1      777,887     1,415,762   1,341,382      74,380        703,507

KFSHRC  39,878      0           0              0              39,878

W1      285,348     624,723     618,374        6,349          278,999